In [2]:
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_sm")

def extract_relations_spacy(text):
    doc = nlp(text)
    relations = []
    
    for token in doc:
        if token.dep_ in ("nsubj", "dobj"):  # Subject-Object relation
            subject = [w.text for w in token.lefts if w.dep_ == "compound"] + [token.text]
            obj = [w.text for w in token.rights if w.dep_ == "compound"] + [token.text]
            relations.append((" ".join(subject), token.head.text, " ".join(obj)))

    return relations


file_path = "news_excerpts_parsed.xlsx"  # Update with actual path
df = pd.read_excel(file_path)

df["Extracted_Relationships"] = df["Text"].apply(extract_relations_spacy)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# ✅ Load Pre-Trained Relationship Extraction Model
model_name = "zhengzangw/bert-relation-extraction"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# ✅ Create an Inference Pipeline
relation_extraction = pipeline("text-classification", model=model, tokenizer=tokenizer)

# ✅ Load Dataset (Excerpts with Extracted Entities)
file_path = "results/generalized_normalized_ner_with_embeddings.xlsx"  # Ensure this contains "Text" & "Normalized_NER_Entities" 
df = pd.read_excel(file_path)

# ✅ Function to Extract Relationships from Text
def extract_entity_relationships(text, entities):
    """
    Given a text excerpt and a list of extracted entities, 
    find relationships between entity pairs using BERT.
    """
    relationships = []
    
    # Generate all possible entity pairs (A, B)
    entity_names = [name for name, _, _ in eval(entities)]  # Extract entity names

    for i in range(len(entity_names)):
        for j in range(i + 1, len(entity_names)):  # Only unique pairs
            entity1, entity2 = entity_names[i], entity_names[j]
            
            # Format input text for BERT
            input_text = f"[Entity1: {entity1}] [Entity2: {entity2}] - {text}"

            # Get relationship prediction
            result = relation_extraction(input_text)
            predicted_relation = result[0]["label"]
            confidence = round(result[0]["score"], 4)

            # Store results
            relationships.append((entity1, entity2, predicted_relation, confidence))
    
    return relationships

# ✅ Apply Relationship Extraction Model to Dataset
df["Extracted_Relationships"] = df.apply(
    lambda row: extract_entity_relationships(row["Text"], row["Normalized_NER_Entities"]),
    axis=1
)

# ✅ Save Relationships to Excel File
df.to_excel("bert_extracted_relationships.xlsx", index=False)

print("✅ Relationship extraction complete! Results saved to 'bert_extracted_relationships.xlsx'.")

/Users/joelleo/Desktop/Uni/Competitions/SMU Datathon/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: zhengzangw/bert-relation-extraction is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [6]:
from relik import Relik
from relik.inference.data.objects import RelikOutput

# Load the pre-trained ReLiK model
relik = Relik.from_pretrained("sapienzanlp/relik-relation-extraction-nyt-large")

# Define your text excerpt
text = "Starbucks violated federal labor law when it increased wages and offered new perks and benefits only to non-union employees, a National Labor Relations Board judge found Thursday. The decision is the latest in a series of NLRB rulings finding that Starbucks has violated labor law in its efforts to stop unions from forming in its coffee shops. The issue at the heart of this case is whether, under current Board law, [Starbucks] was entitled to explicitly reward employees,” for not participating in union activity, “while falsely telling its workers that the federal labor law forced it to take this action,” wrote administrative law judge Mara-Louise Anzalone. “It was not.”"

# Perform relation extraction
relik_out: RelikOutput = relik(text)

# Display the extracted relations
for triplet in relik_out.triplets:
    subject = triplet.subject.text
    relation = triplet.label
    obj = triplet.object.text
    print(f"Subject: {subject}, Relation: {relation}, Object: {obj}")

                ___              __         
               /\_ \      __    /\ \        
 _ __     __   \//\ \    /\_\   \ \ \/'\    
/\`'__\ /'__`\   \ \ \   \/\ \   \ \ , <    
\ \ \/ /\  __/    \_\ \_  \ \ \   \ \ \\`\  
 \ \_\ \ \____\   /\____\  \ \_\   \ \_\ \_\
  \/_/  \/____/   \/____/   \/_/    \/_/\/_/
                                            
                                            



/Users/joelleo/Desktop/Uni/Competitions/SMU Datathon/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2025-02-01 17:09:06,201] [INFO] [relik.inference.annotator.from_pretrained:700] [PID:13735] [RANK:0] Loading Relik from sapienzanlp/relik-relation-extraction-nyt-large
[2025-02-01 17:09:06,204] [INFO] [relik.inference.annotator.from_pretrained:701] [PID:13735] [RANK:0] {
    '_target_': 'relik.inference.annotator.Relik',
    'index': {
        'triplet': {
            '_target_': 'relik.retriever.indexers.inmemory.InMemoryDocumentIndex.from_pretrained',
            'name_or_path': 'sapienzanlp/relik-retriever-small-nyt-document-index',
        },
    },
    'metadata_fields': [],
    'reader': {
        '_target_': 'relik.reader.pytorch_modules.triplet.RelikReaderForTripletExtraction',
        'transformer_model': 'sapienzanlp/relik-reader-deberta-v3-large-nyt',
    },
    'retriever': {
        'triplet': {
            '_target_': 'relik.retriever.pytorch_modules.model.GoldenRetriever',
            'question_encoder': 'sapienzanlp/relik-retriever-small-nyt-question-encoder',
        

In [12]:
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd

# Download NLTK tokenizer
nltk.download('punkt')

# Load relationship extraction model
model_name = "Babelscape/rebel-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define the text excerpt
text = """Starbucks violated federal labor law when it increased wages and offered new perks and benefits only to non-union employees, 
a National Labor Relations Board judge found Thursday. The decision is the latest in a series of NLRB rulings finding that Starbucks 
has violated labor law in its efforts to stop unions from forming in its coffee shops. The issue at the heart of this case is whether, 
under current Board law, [Starbucks] was entitled to explicitly reward employees,” for not participating in union activity, “while 
falsely telling its workers that the federal labor law forced it to take this action,” wrote administrative law judge Mara-Louise Anzalone. “It was not.”"""

# Define the entities
entities = [
    ('Starbucks', 'ORG', 0.9993),
    ('National Labor Relations Board', 'ORG', 0.9994),
    ('NLRB', 'ORG', 0.9989),
    ('National Heritage Board', 'ORG', 0.9894),
    ('Asmara', 'PER', 0.9989),
    ('Louise Anzalone', 'PER', 0.9376)
]

# ✅ Break the text into sentences for better extraction
sentences = sent_tokenize(text)

# Function to extract relationships from each sentence
def extract_relationships_from_sentences(sentences, entities):
    relationships = []
    
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, max_length=512)
        outputs = model.generate(**inputs)

        # Decode output
        relation_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Parse the output into structured relationships
        relation_pairs = relation_text.split(";")
        for relation in relation_pairs:
            words = relation.split()
            if len(words) < 3:
                continue
            
            head, tail, rel_description = words[0], words[-1], " ".join(words[1:-1])

            # Find the closest matching entity in the provided list
            head_entity = next((e[0] for e in entities if head.lower() in e[0].lower()), None)
            tail_entity = next((e[0] for e in entities if tail.lower() in e[0].lower()), None)

            if head_entity and tail_entity:
                relationships.append({
                    "head": head_entity,
                    "tail": tail_entity,
                    "relationship": rel_description,
                    "relationship_type": "unspecified"
                })

    return relationships

# Extract relationships sentence by sentence
relationship_data = extract_relationships_from_sentences(sentences, entities)

# Save results to a DataFrame
df = pd.DataFrame(relationship_data)
df.to_excel("bert_extracted_relationships.xlsx", index=False)

# Print extracted relationships
print("✅ Relationship extraction complete! Results saved to 'bert_extracted_relationships.xlsx'.")
print(df)

[nltk_data] Downloading package punkt to /Users/joelleo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✅ Relationship extraction complete! Results saved to 'bert_extracted_relationships.xlsx'.
Empty DataFrame
Columns: []
Index: []


In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the rebel model
model_name = "wietsedv/bert-base-relation-extraction"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Example usage
text = "Starbucks violated federal labor law when it increased wages and offered new perks and benefits only to non-union employees."
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs)

# Decode the output
relation_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Extracted Relationship:", relation_text)

OSError: wietsedv/bert-base-relation-extraction is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`